<a href="https://colab.research.google.com/github/devansh207/Datasets/blob/master/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
a = []
while(1):
    a.append(1)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
d=pd.read_csv('/content/IMDB Dataset.csv')
d.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
d.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [ ]:
import keras 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Dropout, Embedding, Flatten, LSTM, Bidirectional
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


In [ ]:
print(d.iloc[0])

#check the labels in the dataset
d.sentiment.value_counts()

#converts label into integer values
d['sentiment'] = d.sentiment.astype('category').cat.codes

review       One of the other reviewers has mentioned that ...
sentiment                                             positive
Name: 0, dtype: object


In [ ]:

num_class = len(np.unique(d.sentiment.values))
y = d['sentiment'].values
print("\nThere are a total of " + str(num_class) + " classes.")

# evenly distributed dataset
d.groupby('sentiment').count()


There are a total of 2 classes.


,review
sentiment,
0,25000
1,25000


In [ ]:
 # number of words to be used in the word corpse
maxlen = 1600 #caught off point of the lenght of a sentence

tokenizer = Tokenizer()
#fits the comments on the comments
tokenizer.fit_on_texts(d.review.values)
#changes the tokens into sequence data structures
post_seq= tokenizer.texts_to_sequences(d.review.values)
post_seq_padded= pad_sequences(post_seq, maxlen=maxlen)

In [ ]:
max_features=len(tokenizer.word_index)+1

In [ ]:
print(max_features)

124253


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(post_seq_padded, y, test_size=0.3, random_state=23)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras import optimizers

model = Sequential()
model.add(Embedding(max_features, 100, input_length=maxlen))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])


# prints out summary of model
model.summary()

# saves the model weights
# the train/validation/test split is 26500/8750/15000, it's essential to hold out a decent chunck of unseen data
filepath="weights-simple.hdf5"
checkpointer = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
history = model.fit([X_train], batch_size=32, y=to_categorical(y_train), verbose=1, validation_split=0.25, 
          shuffle=True, epochs=3, callbacks=[checkpointer])


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1600, 100)         12425300  
_________________________________________________________________
dropout_11 (Dropout)         (None, 1600, 100)         0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 128)               84480     
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 258       
Total params: 12,510,038
Trainable params: 12,510,038
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 26250 samples, validate on 8750 samples
Epoch 1/3
26250/26250 [==============================] - 2239s 85ms/step - loss: 0.4299 - acc: 0.7987 - val_loss: 0.2989 - val_acc: 0.8802

Epoch 00001: val_acc improved from -inf to 0.88023, saving model to weights-simple.hdf5
Epoch 2/3
26250/26250 [==============================] - 2229s 85ms/step - loss: 0.2304 - acc: 0.9147 - val_loss: 0.3088 - val_acc: 0.8846

Epoch 00002: val_acc improved from 0.88023 to 0.88457, saving model to weights-simple.hdf5
Epoch 3/3
26250/26250 [==============================] - 2219s 85ms/step - loss: 0.1504 - acc: 0.9453 - val_loss: 0.3378 - val_acc: 0.8830

Epoch 00003: val_acc did not improve from 0.88457


In [ ]:
#get prediction accuarcy for testing dataset 15000 samples
predicted = model.predict(X_test)
predicted_best = np.argmax(predicted, axis=1)
print (accuracy_score(predicted_best, y_test))
predicted=pd.DataFrame(data=predicted)

0.8805333333333333
